In [1]:
import os

In [2]:
# Check present working directory
%pwd

'f:\\Interview_kickstart\\MLOPs_project\\End_to_end_MLOPS_project\\research'

In [5]:
# Change present working directory to thr root directory
os.chdir('../')

In [6]:
%pwd

'f:\\Interview_kickstart\\MLOPs_project\\End_to_end_MLOPS_project'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-21 00:37:21,511: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-21 00:37:21,513: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-21 00:37:21,516: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-21 00:37:21,517: INFO: common: created directory at: artifacts]
[2023-11-21 00:37:21,518: INFO: common: created directory at: artifacts/data_ingestion]
[2023-11-21 00:37:22,887: INFO: 2280763589: artifacts/data_ingestion/data.zip download! with following info: 
Content-Type: text/html; charset=utf-8
X-Robots-Tag: noindex, nofollow, nosnippet
Cache-Control: no-cache, no-store, max-age=0, must-revalidate
Pragma: no-cache
Expires: Mon, 01 Jan 1990 00:00:00 GMT
Date: Mon, 20 Nov 2023 19:07:20 GMT
P3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."
Content-Security-Policy: require-trusted-types-for 'script';report-uri https://csp.withgoogle.com/csp/docs-tt
Referrer-Policy: origin
X-Content-Type-Options: nosniff


BadZipFile: File is not a zip file